# Exploring Inconsistancies Between AllStatesAndTerritories Files

This notebook explores inconsistancies in the *LESO Property Transferred to Participating Agencies* files that are published quarterly on the [DLA LESO Public Data](https://www.dla.mil/DispositionServices/Offers/Reutilization/LawEnforcement/PublicInformation/) website. As of October 2020, three DISP_AllStatesAndTransfers files have been collected for the quarters ending March 31, June 30 and September 30 of 2020.

In [ ]:
#    Libraries used by this notebook.

import pandas as pd
import matplotlib.pyplot as plt
import sys

from IPython.display import Markdown as md

#!python --version    #Python 3.8.5
# sys standard module
#!jupyter --version
# ipython             #ipython 7.18.1
#pd.__version__       #1.1.2

sys.path.insert(0, "..\\..\\scripts\\")
from checksumfunctions import get_file_info
from checksumfunctions import get_file_hash
from notebookfunctions import make_dataframe
from notebookfunctions import count_by_time
from notebookfunctions import differences_by_time
from notebookfunctions import split_demilcodes

#    VARIABLES THAT CAN BE CUSTOMIZED

#    Enter the path to the folder containing all the data files.
path_datafiles = "../../data/"

#    Get the 'LESO Property Transferred to Participating Agencies' file from 
#        Defense Logicstics Agency Law Enforcement Support Office Public Information
#    The original name of the data file should be in the form:
#        DISP_AllStatesAndTerritories_mmddyyyy.xlsx  
#
#    Enter the name of the LESO files to be checked.
LESO1_file = "DISP_AllStatesAndTerritories_03312020.xlsx"
LESO2_file = "DISP_AllStatesAndTerritories_06302020.xlsx"
LESO3_file = "DISP_AllStatesAndTerritories_09302020.xlsx"
#LESO4_file = "DISP_AllStatesAndTerritories_12312020.xlsx"

#    Read the data from the XLSX files.
#    transfer#_dict is a dictionary of all sheets in the LESO_file
#         keys are full state/territory names
#         values are a single dataframe of all transfers for that state/territory
#    The records may be cumulative up to this quarter.
transfers1_dict = pd.read_excel('file:' + path_datafiles + LESO1_file, sheet_name=None)
transfers2_dict = pd.read_excel('file:' + path_datafiles + LESO2_file, sheet_name=None)
transfers3_dict = pd.read_excel('file:' + path_datafiles + LESO3_file, sheet_name=None)
#transfers4_dict = pd.read_excel('file:' + path_datafiles + LESO4_file, sheet_name=None)

#    Flatten each dictionary of states into its own dataframe
transfers1_df = make_dataframe(transfers1_dict, 'Ship Date')
transfers2_df = make_dataframe(transfers2_dict, 'Ship Date')
transfers3_df = make_dataframe(transfers3_dict, 'Ship Date')
#transfers4_df = make_dataframe(transfers4_dict, 'Ship Date')

#    Temporary way to track files the notebook is working with
file_quarters_dict = {'transfers1': (transfers1_df, 'Quarter 1'),
                      'transfers2': (transfers2_df, 'Quarter 2'),
                      'transfers3': (transfers3_df, 'Quarter 3')}

In [ ]:
print('This notebook is comparing the following files: ')
for file_name in [LESO1_file, LESO2_file, LESO3_file]:
    print('%s\t%s\t%s' % get_file_info(path_datafiles + file_name))
    print('\t\t\tMD5\t %s' % get_file_hash(path_datafiles + file_name, 'md5'))
    print('\t\t\tSHA256\t %s' % get_file_hash(path_datafiles + file_name, 'sha256'))

### What is the range of dates on records in each file?

Under the section called *LESO Property Transferred to Participating Agencies*, the DLA LESO Public Data website says this file is "... is the most recent quarterly update of the accountable property held by participating agencies." The LESO files we will be looking at in this notebook have the following 'Ship Date' ranges:

In [ ]:
#    Explore the range of dates in each file.
pd.DataFrame({file_info[1]: [file_info[0].index.to_period('d').min(),
                             file_info[0].index.to_period('d').max()]
              for file_info in file_quarters_dict.values()},index=['Earliest Date in File','Latest Date in File'])

A given DLA LESO AllStatesAndTerritories file may have records that are more recent than the end of the published quarter. These records may or may not be consistant with the files in succeeding quarters as can be seen by exploring the record count for overlapping dates.

In [ ]:
#    Explore the record counts for dates outside the expected quarter by file.
outside_quarter_start = '2020-04-01'
outside_quarter_end = '2020-04-21'

april_df = pd.DataFrame({
                    file_quarters_dict['transfers1'][1]: file_quarters_dict['transfers1'][0].\
                         loc[outside_quarter_start:outside_quarter_end].groupby('Ship Date')['Ship Date'].count(),
                    file_quarters_dict['transfers2'][1]: file_quarters_dict['transfers2'][0].\
                         loc[outside_quarter_start:outside_quarter_end].groupby('Ship Date')['Ship Date'].count(),
                    file_quarters_dict['transfers3'][1]: file_quarters_dict['transfers3'][0].\
                         loc[outside_quarter_start:outside_quarter_end].groupby('Ship Date')['Ship Date'].count()
                    }).fillna(0).astype(int)
april_df.index = april_df.index.to_period('d')

april_differences = april_df.diff(axis=1).iloc[:, 1:]
april_differences.columns = ['Quarter 2 - Quarter 1','Quarter 3 - Quarter 2']
april_differences[(april_differences['Quarter 2 - Quarter 1'] != 0) | 
                  (april_differences['Quarter 3 - Quarter 2'] != 0)]
april_differences.index = april_differences.index.day.astype(str)

#    Make a graph
fig = plt.figure(constrained_layout=False, figsize=(18,6))
gs1 = fig.add_gridspec(nrows=1, ncols=3)
fig_ax1 = fig.add_subplot(gs1[0,0:2])
fig_ax2 = fig.add_subplot(gs1[0,2:])
april_df.plot.bar(ax=fig_ax1,rot=90, legend=True, ylabel='Total Records',
                  title='Daily Record Count by File, April 2020')
april_differences.plot.line(ax=fig_ax2, legend=True,
                            xticks=(range(len(april_differences.index))), yticks=(range(-2,16,2)),
                            xlabel='Day in April',ylabel='Difference in Count',
                            title='Changes Between Quarters, April 2020')
plt.show()

### How many total records are there in each file?

In [ ]:
#    Explore total records in each file for x in file_quarters_dict.values()
transfer_count_series = pd.Series({'Quarter 1': file_quarters_dict['transfers1'][0].shape[0],
                                   'Quarter 2': file_quarters_dict['transfers2'][0].shape[0],
                                   'Quarter 3': file_quarters_dict['transfers3'][0].shape[0]})

#    Make a graph
fig = plt.figure(constrained_layout=False, figsize=(18,8))

gs1 = fig.add_gridspec(nrows=1, ncols=10)
fig_ax1 = fig.add_subplot(gs1[0,:6])
transfer_count_series.plot(ax=fig_ax1, figsize=(12, 4), ylabel='Total Records',
                                                             xticks=(range(3)), style='.-',
                                                             title='Trend for Total Records in Each Quarterly File')

ax2 = fig.add_subplot(gs1[0,8])
font_size=12
bbox=[0, 0, 5, .25]
ax2.axis('off')
table_ax1 = ax2.table(cellText = [transfer_count_series.values], rowLabels = ['Record Count'],
                      bbox=bbox, colLabels=transfer_count_series.index.astype(str))
table_ax1.auto_set_font_size(False)
table_ax1.set_fontsize(font_size)

The number of records, or DLA LESO items held by law enforcement agecies, is declining from file to file despite the growing time ranges and, presumably, requests for materials. Documentation at the [DLA LESO Public Data FAQ](https://www.dla.mil/DispositionServices/Offers/Reutilization/LawEnforcement/ProgramFAQs.aspx) states that *LESO Property Transferred to Participating Agencies* files are snapshots of the DLA LESO inventory of items held by law enforcement agencies at the end of a given quarter. It states that after 1 year, non-controlled items are removed from DLA LESO inventory and become the property of local law enforcement agencies.   

### How do the record counts from each file compare when split by controlled and non-controlled values?

There are two definitions found for controlled items. The definition at the DLA LESO Public Data website says [controlled](https://www.dla.mil/Portals/104/Documents/DispositionServices/LESO/DISP_ControlledPropertyDefinition_062019.pdf) items have a 'DEMIL Code' of B, C, D, E, F or G as well as those with a 'DEMIL Code' Q combined with a 'DEMIL IC' value of 3. The relevant U. S. code, [10 U.S.C. &sect; 2576a(f)](https://uscode.house.gov/view.xhtml?req=granuleid:USC-prelim-title10-section2576a&num=0&edition=prelim), defines controlled items as having a 'DEMIL Code' of B, C, D, E, G, and Q. For the purpose of this notebook, the first definition is used. Records that are not in the controlled group are considered non-controlled.   

In [ ]:
#    Explore record count when split by controlled/non-controlled values
get_Q_controlled = lambda x: x[(x['DEMIL Code'] == 'Q') & (x['DEMIL IC'] == 3)].groupby('DEMIL Code')['DEMIL Code'].count()
get_Q_noncontrolled = lambda x: x[(x['DEMIL Code'] == 'Q') & (x['DEMIL IC'] != 3)].groupby('DEMIL Code')['DEMIL Code'].count()

demilcode_qNON_count = pd.DataFrame([get_Q_noncontrolled(x[0]) for x in file_quarters_dict.values()]).T
demilcode_qNON_count.columns = ['Quarter 1', 'Quarter 2', 'Quarter 3']
demilcode_qNON_count.index = ['QNON']

demilcode_q3_count = pd.DataFrame([get_Q_controlled(x[0]) for x in file_quarters_dict.values()]).T
demilcode_q3_count.columns = ['Quarter 1', 'Quarter 2', 'Quarter 3']
demilcode_q3_count.index = ['Q3']

demilcode_count = pd.concat([x[0].groupby('DEMIL Code')['Ship Date'].count() for x in file_quarters_dict.values()], axis=1)
demilcode_count.columns = [x[1] for x in file_quarters_dict.values()]
demilcode_count = pd.concat([demilcode_count,demilcode_q3_count,demilcode_qNON_count])

#    Make a graph
fig = plt.figure(constrained_layout=False, figsize=(18,8))

gs1 = fig.add_gridspec(nrows=1, ncols=5)
fig_ax1 = fig.add_subplot(gs1[0,:3])
demilcode_count.plot.bar(ax=fig_ax1, rot=0, figsize=(12,6), legend=True, xlabel='DEMIL Code', ylabel='Total Records',
                  title='Total Record Count for Each File by DEMIL Code')

ax2 = fig.add_subplot(gs1[0,4])
font_size=12
bbox=[0, 0, 2, .75]
ax2.axis('off')
table_ax1 = ax2.table(cellText = demilcode_count.values, rowLabels = demilcode_count.index,
                      bbox=bbox, colLabels=demilcode_count.columns, label='Actual Counts')
table_ax1.auto_set_font_size(False)
table_ax1.set_fontsize(font_size)

In [ ]:
#    Explore differences between counts of controlled/non-controlled values compared total records
diff_total = [(transfer_count_series['Quarter 1'] - transfer_count_series['Quarter 2']),
             (transfer_count_series['Quarter 2'] - transfer_count_series['Quarter 3']),
             (transfer_count_series['Quarter 1'] - transfer_count_series['Quarter 3'])]
diff_noncon = [(demilcode_count.loc[['A','QNON'], 'Quarter 1'].sum() - demilcode_count.loc[['A','QNON'], 'Quarter 2'].sum()),
               (demilcode_count.loc[['A','QNON'], 'Quarter 2'].sum() - demilcode_count.loc[['A','QNON'], 'Quarter 3'].sum()),
              (demilcode_count.loc[['A','QNON'], 'Quarter 1'].sum() - demilcode_count.loc[['A','QNON'], 'Quarter 3'].sum())]
diff_con = [(demilcode_count.loc[['B','C','D','E','F','Q3'], 'Quarter 1'].sum() - demilcode_count.loc[['B','C','D','E','F','Q3'], 'Quarter 2'].sum()),
           (demilcode_count.loc[['B','C','D','E','F','Q3'], 'Quarter 2'].sum() - demilcode_count.loc[['B','C','D','E','F','Q3'], 'Quarter 3'].sum()),
           (demilcode_count.loc[['B','C','D','E','F','Q3'], 'Quarter 1'].sum() - demilcode_count.loc[['B','C','D','E','F','Q3'], 'Quarter 3'].sum())]
pd.DataFrame({'Total Records': diff_total,
              'Non-Controlled Records': diff_noncon,
              'Controlled Records': diff_con},index=['Quarter 1 minus Quarter 2', 
                                                     'Quarter 2 minus Quarter 3',
                                                     'Quarter 1 minus Quarter 3'])


Looking at the difference between counts of controlled and non-controlled records in the different files, it is clear that the loss of records between files from each quarter is not accounted for by the removal of non-controlled records from the DLA LESO inventory. 

### Are there patterens in the counts of controlled vs. non-controlled records since 2000?

In [ ]:
#    Split into controlled/non-controlled dataframes for each file
controlled1, noncontrolled1 = split_demilcodes(file_quarters_dict['transfers1'][0])
controlled2, noncontrolled2 = split_demilcodes(file_quarters_dict['transfers2'][0])
controlled3, noncontrolled3 = split_demilcodes(file_quarters_dict['transfers3'][0])

In [ ]:
#    Explore patterns in controlled dataframes
diff_trans1c_and_trans2c = differences_by_time('Y', '2000-01-01', '2020-09-30', controlled1, controlled2)
diff_trans2c_and_trans3c = differences_by_time('Y', '2000-01-01', '2020-09-30', controlled2, controlled3)

compare_controlled = diff_trans1c_and_trans2c.merge(diff_trans2c_and_trans3c, how='outer',
                                               left_index=True, right_index=True, suffixes=['_first','_second'])
compare_controlled.columns = ['Quarter 1(compared with Quarter 2)','Quarter 2(compared with Quarter 1)',
                              'Quarter 2(compared with Quarter 3)','Quarter 3(compared with Quarter 2)']
compare_controlled = compare_controlled.fillna(0).astype(int)
compare_controlled.index = compare_controlled.index.to_period('y')

#    Make a graph
title_str = 'Differences in Controlled Record Count Since 2000 (excluding years with no differences)'
ax = compare_controlled.plot.bar(rot=90, figsize=(18,8), legend=True, xlabel='Year', ylabel='Total Records',
                                 title=title_str)

In [ ]:
#    Explore patterns in controlled dataframes
diff_trans1nc_and_trans2nc = differences_by_time('Y', '2000-01-01', '2020-09-30', noncontrolled1, noncontrolled2)
diff_trans2nc_and_trans3nc = differences_by_time('Y', '2000-01-01', '2020-09-30', noncontrolled2, noncontrolled3)

compare_noncontrolled = diff_trans1nc_and_trans2nc.merge(diff_trans2nc_and_trans3nc, how='outer',
                          left_index=True, right_index=True, suffixes=['_first','_second'])
compare_noncontrolled.columns = ['Quarter 1(compared with Quarter 2)','Quarter 2(compared with Quarter 1)',
                                 'Quarter 2(compared with Quarter 3)','Quarter 3(compared with Quarter 2)']
compare_noncontrolled = compare_noncontrolled.fillna(0).astype(int)
compare_noncontrolled.index = compare_noncontrolled.index.to_period('y')

#    Make a graph
title_str = 'Differences in Non-Controlled Record Count Since 2000 (excluding years with no differences)'
ax = compare_noncontrolled.plot.bar(rot=90, figsize=(18,8), legend=True, xlabel='Year', ylabel='Total Records',
                                    title=title_str)

As expected, 2019 shows a big decrease in non-controlled records. In each file, there is some change in both controlled and non-controlled records beyond one year ago. No explanation for why this happens has yet been found. 

### Addendum: A Closer Look at Inconsistancies in a Year

Exploring the month-by-month differences in record count for a particular year shows records are not only disappearing, but being added to the files each quarter. Therefore, data from a file downloaded for one quarter may not be consistant with data downloaded in another quarter even for dates earlier than the previous year.   

#### A Closer Look at 2014 

In [ ]:
#    Explore patterns in controlled records of dataframes for 2018
month_diff_trans1c_and_trans2c = differences_by_time('M', '2014-01-01', '2014-12-31', controlled1, controlled2)
month_diff_trans2c_and_trans3c = differences_by_time('M', '2014-01-01', '2014-12-31', controlled2, controlled3)

month_compare_controlled = month_diff_trans1c_and_trans2c.\
                                    merge(month_diff_trans2c_and_trans3c, how='outer',
                                          left_index=True, right_index=True, suffixes=['_first','_second'])
month_compare_controlled.columns = ['Quarter 1(compared with Quarter 2)','Quarter 2(compared with Quarter 1)',
                                    'Quarter 2(compared with Quarter 3)','Quarter 3(compared with Quarter 2)']
month_compare_controlled = month_compare_controlled.fillna(0).astype(int)
month_compare_controlled.index = month_compare_controlled.index.to_period('m')

#    Make a graph
title_str = 'Differences in Controlled Record Count for 2014 \
(excluding months with no differences)'
ax = month_compare_controlled.plot.bar(rot=90, figsize=(12,6), legend=True,
                                       xlabel='Month', ylabel='Total Records',
                                       title=title_str)

In [ ]:
#    Explore patterns in non-controlled records of dataframes for 2018
month_diff_trans1nc_and_trans2nc = differences_by_time('M', '2014-01-01', '2014-12-31', noncontrolled1, noncontrolled2)
month_diff_trans2nc_and_trans3nc = differences_by_time('M', '2014-01-01', '2014-12-31', noncontrolled2, noncontrolled3)

month_compare_noncontrolled = month_diff_trans1nc_and_trans2nc.\
                                    merge(month_diff_trans2nc_and_trans3nc, how='outer',
                                          left_index=True, right_index=True, suffixes=['_first','_second'])
month_compare_noncontrolled.columns = ['Quarter 1(compared with Quarter 2)','Quarter 2(compared with Quarter 1)',
                                       'Quarter 2(compared with Quarter 3)','Quarter 3(compared with Quarter 2)']
month_compare_noncontrolled = month_compare_noncontrolled.fillna(0).astype(int)
month_compare_noncontrolled.index = month_compare_noncontrolled.index.to_period('m')

#    Make a graph
title_str = 'Differences in Non-Controlled Record Count for 2014 \
(excluding months with no differences)'
ax = month_compare_noncontrolled.plot.bar(rot=90, figsize=(12,6), legend=True,
                                          xlabel='Month', ylabel='Total Records',
                                          title=title_str)

#### A Closer Look at 2018 

In [ ]:
#    Explore patterns in controlled records of dataframes for 2018
month_diff_trans1c_and_trans2c = differences_by_time('M', '2018-01-01', '2018-12-31', controlled1, controlled2)
month_diff_trans2c_and_trans3c = differences_by_time('M', '2018-01-01', '2018-12-31', controlled2, controlled3)

month_compare_controlled = month_diff_trans1c_and_trans2c.\
                                    merge(month_diff_trans2c_and_trans3c, how='outer',
                                          left_index=True, right_index=True, suffixes=['_first','_second'])
month_compare_controlled.columns = ['Quarter 1(compared with Quarter 2)','Quarter 2(compared with Quarter 1)',
                                    'Quarter 2(compared with Quarter 3)','Quarter 3(compared with Quarter 2)']
month_compare_controlled = month_compare_controlled.fillna(0).astype(int)
month_compare_controlled.index = month_compare_controlled.index.to_period('m')

#    Make a graph
title_str = 'Differences in Controlled Record Count for 2018 \
(excluding months with no differences)'
ax = month_compare_controlled.plot.bar(rot=90, figsize=(12,6), legend=True,
                                       xlabel='Month', ylabel='Total Records',
                                       title=title_str)

In [ ]:
#    Explore patterns in non-controlled records of dataframes for 2018
month_diff_trans1nc_and_trans2nc = differences_by_time('M', '2018-01-01', '2018-12-31', noncontrolled1, noncontrolled2)
month_diff_trans2nc_and_trans3nc = differences_by_time('M', '2018-01-01', '2018-12-31', noncontrolled2, noncontrolled3)

month_compare_noncontrolled = month_diff_trans1nc_and_trans2nc.\
                                    merge(month_diff_trans2nc_and_trans3nc, how='outer',
                                          left_index=True, right_index=True, suffixes=['_first','_second'])
month_compare_noncontrolled.columns = ['Quarter 1(compared with Quarter 2)','Quarter 2(compared with Quarter 1)',
                                       'Quarter 2(compared with Quarter 3)','Quarter 3(compared with Quarter 2)']
month_compare_noncontrolled = month_compare_noncontrolled.fillna(0).astype(int)
month_compare_noncontrolled.index = month_compare_noncontrolled.index.to_period('m')

#    Make a graph
title_str = 'Differences in Non-Controlled Record Count for 2018 \
(excluding months with no differences)'
ax = month_compare_noncontrolled.plot.bar(rot=90, figsize=(12,6), legend=True,
                                          xlabel='Month', ylabel='Total Records',
                                          title=title_str)